In [36]:
import pandas as pd
from glob import glob
import os
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch 
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import json
import requests
import numpy as np
import pandas as pd
import stanza
import scipy
import pickle
import sklearn
from collections import defaultdict,deque
from geopy.geocoders import Nominatim
from geopy import distance
from statistics import mode
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from word2number import w2n
import ast

In [37]:
ner_tokenizer = AutoTokenizer.from_pretrained("datummd/NCBI_BC5CDR_disease",model_max_length=512)
ner_model = AutoModelForTokenClassification.from_pretrained("datummd/NCBI_BC5CDR_disease")
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer,grouped_entities = True)

/home/marwan/event/lib/python3.8/site-packages/transformers/pipelines/token_classification.py:135: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [38]:
qa_pipeline = pipeline('question-answering', model = "marwanimroz18/best_model", tokenizer="marwanimroz18/best_model")

In [39]:
import pandas as pd
column_names=["Index","Original_Article",'Translated_Article','Events','QA_data','User']

df = pd.read_csv('translated_data(2).csv',names=column_names)
df

,Index,Original_Article,Translated_Article,Events,QA_data,User
0,0,ग्वालियर में 183 संदिग्ध राेगियाें की हुई जांच...,183 suspected patients were investigated in Gw...,"[{'Disease': 'COVID-19', 'Location': 'Gwalior'...",[{'Question': 'How many suspected COVID-19 pat...,Marwan
1,1,मोरबी ब्रिज हादसा : मरम्मत करने वाली कंपनी के ...,NaN,[],"[{'Question': ""How many people were killed whe...",Saba
2,2,जोधपुर में गैस सिलेंडर रिफिलिंग के दौरान लगी आ...,Fire broke out during gas cylinder refilling i...,[],[{'Question': 'Which place of Jodhpur was shak...,Saba
3,3,Increasing cases of lumpy skin disease in Utta...,Increasing cases of lumpy skin disease in Utta...,"[{'Disease': 'Lumpy skin disease', 'Location':...",[{'Question': 'In which place Lumpy skin disea...,Saba
4,4,झारखंड के कस्तूरबा स्कूल में फूड पॉइजनिंग से 2...,"In Jharkhand's Kasturba School, 28 girls fell ...",[],[{'Question': 'How many girls fall ill due to ...,Saba
...,...,...,...,...,...,...
752,1455,"Snake found in midday meal, 98 children fall i...","Snake found in midday meal, 98 children fall i...",[],[{'Question': 'How many children fell ill afte...,Marwan
753,1456,"More than 4,800 'victims' of Church sex abuse ...","More than 4,800 'victims' of Church sex abuse ...",[],[{'Question': 'How many people have been victi...,Marwan
754,1457,"What is Lumpy Skin disease, affecting lakhs of...","What is Lumpy Skin disease, affecting lakhs of...","[{'Disease': 'Lumpy Skin Disease', 'Location':...",[{'Question': 'Which disease was detected in t...,Marwan
755,1458,Five cattle die in fire : The Tribune India . ...,Five cattle die in fire : The Tribune India . ...,[],[{'Question': 'How many cattle died in the fir...,Marwan


In [40]:
df['Events'] = df['Events'].apply(lambda x: eval(x))

In [41]:

net_df = df[df['Events'].apply(lambda x: len(x)>0)]

In [42]:
net_df['User'].value_counts()

User
Marwan    198
Saba      196
Name: count, dtype: int64

In [43]:
len(net_df)

394

In [44]:
# This is a list of disease synonyms. More synonyms can be added to capture those disease if the NER model misses them.
key_list = ["covid-19","coronavirus","sars-ncov-2","malaria","dengue","fever"]
def keyword_spotter(article,key_list):
    '''Performs keyword spotting on the article for the keys in the key_list'''
    spots  =[]
    for key in key_list:
        if article.lower().find(key)!=-1:
            spots.append(key)
    return spots

def get_disease(article,ner_pipeline):
    '''The function returns the diseases found by keyword spotting and by Disease NER model. It returns disease in the form of string. Individual string saperated by commas.
    It also outputs the span of all the occurances of those disease keywords in the article. These spans will  later be used to filter-out redundant answers (eg. 19 from covid-19)'''
    diseases = []
    disease_str = ""
    spans = []
    ner_results = ner_pipeline(article) #gets diseases from disease-ner model
    dis_list = sorted(ner_results,key = lambda x : x["start"])
    new_dis_list = []
    if len(dis_list)!=0:
        print("Dis_list")
        print(dis_list)
        prev = dis_list[0]
        print("prev")
        print(prev)
        for dis in dis_list[1:]:
            print(prev)
            if prev!=None and dis["start"]==prev["end"] and dis["word"].startswith("##"):
                new_word = (prev["word"]+dis["word"]).replace("##","")
                new_dis_list.append(new_word)
                prev = None
            elif(prev==None):
                break
            
            else:
                print(prev)
                new_dis_list.append(prev["word"])
                prev = dis
        print(prev)
        if prev!=None:
            new_dis_list.append(dis_list[-1]["word"])
        diseases = new_dis_list
    keyword_spots = keyword_spotter(article,key_list) #get diseases from keyword spotting
    diseases.extend(keyword_spots) 
    for disease in diseases: # get the span of those diasease in the article. (start_index and end_index)
        res = [(i,i+len(disease)-1) for i in range(len(article)) if article.lower().startswith(disease, i)]
        spans.extend(res)
    if len(diseases)!=0: 
        diseases = list(set(diseases)) #Remove all repeated diseases and convert to a single string.
        disease_str = ",".join(map(str,diseases))
    return disease_str,spans


In [45]:
# def NER(message, entity_name):
#     """
#     NER Function, to exrtact entity from message
    
#     args    : 1) message     : body of message  (type:str)
#               2) entity_name : type of entity to extract from -> (date / number / city) (type:str) 

#     returns : 1) entities    :  (type:list of tuple)
#                 list of tuples of entities extracted , each tuple has 2 elements (original text, entity)
#                 original text is the exact text from the messsage and entity is the entity that it maps to.
#     """

#     message = message.replace("(","").replace(")","").replace("'s","")
    

#     # helper dict to form url
#     url_map = {'date':'date', 
#                'city':'text', 
#                'number':'number'}

#     payload = { 'message': message,
#                 'entity_name': entity_name,
#                 'structured_value': None,
#                 'fallback_value': None,
#                 'bot_message': None,
#                 'source_language': 'en'}


#     # load confg file (edit config.py to make changes
#     # print(host_PORT)
 

#     if entity_name == 'city':
#         url = "http://localhost:8082" + '/v1/{}/?&entity_name={}&source_language=en'.format(url_map[entity_name],entity_name)
#         # print(url)

#     elif entity_name in ['date','number']:
#         url = "http://localhost:8082" + '/v2/{}/?&entity_name={}&source_language=en'.format(url_map[entity_name],entity_name)
        
#     # API Call 
#     response = requests.get(url, params=payload)
#     # print(response)
#     if response.status_code != 200:
#         print("None")
#         return None
    
#     response = response.json()['data']
#     # print(response)

#     # formatting returned value to return list of tuples where we have 
#     # a a tuple for each entity like tuple(original_text,entity)    
    
#     entities = []
#     if response:
#         for entity in response:

#             if entity_name == 'city':
#                 # item_1 = entity['original_text'].replace(",","").replace(".","")
#                 item_1 = entity['original_text'].replace(",","").replace(".","").replace("[","").replace("]","").replace("(","").replace(")","").replace(";","").replace(":","")
#                 item_2 = entity['entity_value']['value']
#                 if item_1!="risen" and item_1!="up":
#                     entities.append(tuple((item_1,item_2)))

#             elif entity_name == 'date':
#                 if entity['entity_value']['value']['type'] == 'date':
#                     dd = entity['entity_value']['value']['dd']
#                     mm = entity['entity_value']['value']['mm']
#                     yy = entity['entity_value']['value']['yy']

#                     item_1 = entity['original_text'].replace(",","").replace(".","").lower()
#                     item_2 = datetime(year=yy,month=mm,day=dd)
#                     entities.append(tuple((item_1,item_2)))

#             elif entity_name == 'number':

#                 item_1 = entity['original_text'].replace(",","").replace(".","").lower()
#                 item_2 = entity['entity_value']['value']
#                 entities.append(tuple((item_1,item_2)))
        

#     return entities


# def get_location_data(locations):
#     """
#     takes list of locaitons and returns list of dictionaries 
#     where each dictionary has the keys of (name,address,longitude,latitude)
#     for each of the locations.

#     args    : 1) locations (type:list of str)

#     returns : 1) location_data (type:list of dict) 
#                     for each location, we have a dict with the following keys 
#                     location_data[location] = {'name' : officail name, 
#                                                'address': heirarchichal address, 
#                                                'latitude': latitude, 
#                                                'longitude':longitude}
#     """

#     geolocator = Nominatim(user_agent="loc_to_latlon")
#     location_data = defaultdict(dict)

#     for location in locations:
#         loc_object = geolocator.geocode(location)

#         if loc_object != None:
#             location_data[location] = {'name' : location, 
#                                     'address': loc_object.address, 
#                                     'latitude':loc_object.latitude, 
#                                     'longitude':loc_object.longitude}
#         else:
#             location_data[location] = None
                                   
#     return location_data


# location = NER("Mumbai: 25% drop in OPD patients as resident docs go on strike ","city")
# # print(location)
def NER(message, entity_name):
    """
    NER Function, to exrtact entity from message
    
    args    : 1) message     : body of message  (type:str)
              2) entity_name : type of entity to extract from -> (date / number / city) (type:str) 

    returns : 1) entities    :  (type:list of tuple)
                list of tuples of entities extracted , each tuple has 2 elements (original text, entity)
                original text is the exact text from the messsage and entity is the entity that it maps to.
    """

    message = message.replace("(","").replace(")","").replace("'s","")
    

    # helper dict to form url
    url_map = {'date':'date', 
               'city':'text', 
               'number':'number'}

    payload = { 'message': message,
                'entity_name': entity_name,
                'structured_value': None,
                'fallback_value': None,
                'bot_message': None,
                'source_language': 'en'}


    # load confg file (edit config.py to make changes)
    with open('config.json') as f:
        configs = json.load(f)
    f.close()
    
    
    # Host Address configuration 
    host_PORT = configs['chatbot_NER_PORT']
    # print(host_PORT)
 

    if entity_name == 'city':
        url = "http://localhost:8081" + '/v1/{}/?&entity_name={}&source_language=en'.format(url_map[entity_name],entity_name)
        # print(url)

    elif entity_name in ['date','number']:
        url = "http://localhost:8081" + '/v2/{}/?&entity_name={}&source_language=en'.format(url_map[entity_name],entity_name)
        
    # API Call 
    response = requests.get(url, params=payload)
    # print(response)
    if response.status_code != 200:
        print("None")
        return None
    
    response = response.json()['data']
    # print(response)

    # formatting returned value to return list of tuples where we have 
    # a a tuple for each entity like tuple(original_text,entity)    
    
    entities = []
    if response:
        for entity in response:

            if entity_name == 'city':
                # item_1 = entity['original_text'].replace(",","").replace(".","")
                item_1 = entity['original_text'].replace(",","").replace(".","").replace("[","").replace("]","").replace("(","").replace(")","").replace(";","").replace(":","")
                item_2 = entity['entity_value']['value']
                if item_1!="risen" and item_1!="up":
                    entities.append(tuple((item_1,item_2)))

            elif entity_name == 'date':
                if entity['entity_value']['value']['type'] == 'date':
                    dd = entity['entity_value']['value']['dd']
                    mm = entity['entity_value']['value']['mm']
                    yy = entity['entity_value']['value']['yy']

                    item_1 = entity['original_text'].replace(",","").replace(".","").lower()
                    item_2 = datetime(year=yy,month=mm,day=dd)
                    entities.append(tuple((item_1,item_2)))

            elif entity_name == 'number':

                item_1 = entity['original_text'].replace(",","").replace(".","").lower()
                item_2 = entity['entity_value']['value']
                entities.append(tuple((item_1,item_2)))
        

    return entities


def get_location_data(locations):
    """
    takes list of locaitons and returns list of dictionaries 
    where each dictionary has the keys of (name,address,longitude,latitude)
    for each of the locations.

    args    : 1) locations (type:list of str)

    returns : 1) location_data (type:list of dict) 
                    for each location, we have a dict with the following keys 
                    location_data[location] = {'name' : officail name, 
                                               'address': heirarchichal address, 
                                               'latitude': latitude, 
                                               'longitude':longitude}
    """

    geolocator = Nominatim(user_agent="loc_to_latlon")
    location_data = defaultdict(dict)

    for location in locations:
        loc_object = geolocator.geocode(location)

        if loc_object != None:
            location_data[location] = {'name' : location, 
                                    'address': loc_object.address, 
                                    'latitude':loc_object.latitude, 
                                    'longitude':loc_object.longitude}
        else:
            location_data[location] = None
                                   
    return location_data


location = NER("Mumbai: 25% drop in OPD patients as resident docs go on strike ","city")
print(location)

[('mumbai', 'Mumbai')]


In [46]:
def check_if_brackets(article,span):
    r_t = min(5,len(article)-span[1])
    l_t  = min(5,span[0])
    # print(article[span[1]:span[1]+r_t-1])
    if r_t>0 and (article[span[1]:span[1]+r_t].find("]")!=-1 or article[span[1]:span[1]+r_t].find(")")!=-1) and l_t>0 and (article[span[0]-l_t:span[0]].find("[")!=-1 or article[span[0]-l_t:span[0]].find("(")!=-1):
        return True
    return False

def check_if_other_location(article,span,spans):
    r_t = min(5,len(article)-span[1])
    l_t  = min(5,span[0])
    if r_t>0 and (article[span[1]:span[1]+r_t].find(":")!=-1 or article[span[1]:span[1]+r_t].find(";")!=-1):
        for s in spans:
            if s[0]<=span[1]+r_t and span[1]<=s[0]:
                return True
    # r_t = min(2,len(article)-span[1])
    # l_t  = min(2,span[0])
    # if r_t>0:
    #     for s in spans:
    #         if s[0]<=span[1]+r_t and span[1]<=s[0]:
    #             return True

    # if l_t>0:
    #     for s in spans:
    #         if s[1]>=span[0]-l_t and span[0]>s[1]:
    #             return True
    return False

def check_in_span_of_other(i,spans):
    for s in spans:
        if i>=s[0] and i<s[1]:
            return True

    return False


def preprocess_location(locations,article):
    locations_found = [x[0] for x in locations]
    
    spans = []
    new_article = ""
    for location in locations_found: # get the span of those diasease in the article. (start_index and end_index)
        res = [(i,i+len(location)) for i in range(len(article)) if article.lower().startswith(location, i)]
        spans.extend(res)
    # print(spans)
    removing_spans = []
    for span in spans:
        # print(article[span[0]:span[1]])
        if check_if_brackets(article,span):
            removing_spans.append(span)
        if check_if_other_location(article,span,spans):
            removing_spans.append(span)
    for i in range(len(article)):
        if check_in_span_of_other(i,removing_spans)!=True:
            new_article+=article[i]
    new_article = new_article.replace("India News","").replace("News India","")
    return new_article


article ="Mumbai: 25% drop in OPD patients as resident docs go on strike "
location = NER(article,"city")
article= preprocess_location(location,article)


In [47]:
def make_questions(dates,locations,disease):
    '''Returns the questions formed by using the dates, locations and disease
    INPUTS:
    dates: a list of dates tuple (date,date_pretty)
    locations: a list of locations (tuple) in a form of tuple. [(location_a, location_a_written_properly)] althoug it is a list of tuple, it should contain only one tuple. For eg. [("delhi","New Delhi")]
    disease: A disease in the form of string '''
    if dates==None:
        dates = []
    if locations==None:
        locations= []
    dates = [x[0] for x in dates] # get the list of dates from the list of date tuple 
    locations = [x[0] for x in locations]
    # locations = [locations[0],locations[0].title()] #get list of locations from a list of location tuple (it should always be of length 1)
    new_cases = []
    total_cases = []
    new_deaths = []
    total_deaths = []
    if len(locations)!=0 and len(dates)!=0: #set of questions when both location and date is present
        for location in locations:
            for date in dates:
                # new cases
                new_cases.append("How many new {} cases were reported in {} on {}".format(disease,location,date))
                new_cases.append("How many {} cases were reported in {} on {}".format(disease,location,date))
                new_cases.append("How many new {} cases were reported in {}".format(disease,location))
                new_cases.append("How many new {} cases were reported in {} in the last 24 hours".format(disease,location))
                new_cases.append("How many fresh {} cases were reported in {} on {}".format(disease,location,date))
                new_cases.append("How many fresh {} cases were reported in {}".format(disease,location))
                new_cases.append("How many fresh {} cases were reported in {} in the last 24 hours".format(disease,location))
                new_cases.append("How many new {} infections were reported in {} on {}?".format(disease,location,date))
                new_cases.append("How many new {} infections were reported in {}".format(disease,location))
                new_cases.append("How many fresh {} infections were reported in {} on {}?".format(disease,location,date))
                new_cases.append("How many fresh {} infections were reported in {}".format(disease,location))
                new_cases.append("How many {} cases were reported in {} in 24 hours".format(disease,location))


                #new deaths
                new_deaths.append("How many new {} deaths were reported in {} on {}".format(disease,location,date))
                new_deaths.append("How many {} deaths were reported in {} on {}".format(disease,location,date))
                new_deaths.append("How many new {} deaths were reported in {}".format(disease,location))
                new_deaths.append("How many new {} deaths were reported in {} in the last 24 hours".format(disease,location))
                new_deaths.append("How many fresh {} deaths were reported in {} on {}".format(disease,location,date))
                new_deaths.append("How many fresh {} deaths were reported in {}".format(disease,location))
                new_deaths.append("How many fresh {} deaths were reported in {} in the last 24 hours".format(disease,location))
                new_deaths.append("How many new deaths due to {} were reported in {}".format(disease,location))   
                new_deaths.append("How many {} deaths were reported in {} in 24 hours".format(disease,location))


                #total cases
                total_cases.append("How many total {} cases were reported in {}".format(disease,location))
                total_cases.append("What is the total number of {} cases reported in {}".format(disease,location))
                total_cases.append("How many total cases of {} were reported in {}".format(disease,location))
                total_cases.append("What is the total tally of {} cases reported in {}".format(disease,location))

                #total deaths
                total_deaths.append("How many total {} deaths were reported in {}".format(disease,location))
                total_deaths.append("How many total deaths due to {} were reported in {}".format(disease,location))
                total_deaths.append("What is the total number of deaths due to {} in {}".format(disease,location))
                total_deaths.append("How many total deaths of {} were reported in {}".format(disease,location))
                total_deaths.append("What is the total tally of {} deaths in {}".format(disease,location))
    if len(locations)!=0 and len(dates)==0: #set of questions when only location is present and no date is present
        for location in locations:
            # new cases
            new_cases.append("How many new {} cases were reported in {}".format(disease,location))
            new_cases.append("How many new {} cases were reported in {} in the last 24 hours".format(disease,location))
            new_cases.append("How many fresh {} cases were reported in {}".format(disease,location))
            new_cases.append("How many fresh {} cases were reported in {} in the last 24 hours".format(disease,location))
            new_cases.append("How many new {} infections were reported in {}".format(disease,location))
            new_cases.append("How many fresh {} infections were reported in {}".format(disease,location))
            new_cases.append("How many {} cases were reported in {} in 24 hours".format(disease,location))
            


            #new deaths
            new_deaths.append("How many new {} deaths were reported in {}".format(disease,location))
            new_deaths.append("How many new {} deaths were reported in {} in the last 24 hours".format(disease,location))
            new_deaths.append("How many fresh {} deaths were reported in {}".format(disease,location))
            new_deaths.append("How many fresh {} deaths were reported in {} in the last 24 hours".format(disease,location))
            new_deaths.append("How many new deaths due to {} were reported in {}".format(disease,location))          
            new_deaths.append("How many {} deaths were reported in {} in 24 hours".format(disease,location))

            #total cases
            total_cases.append("How many total {} cases were reported in {}".format(disease,location))
            total_cases.append("What is the total number of {} cases reported in {}".format(disease,location))
            total_cases.append("How many total cases of {} were reported in {}".format(disease,location))
            total_cases.append("What is the total tally of {} cases reported in {}".format(disease,location))

            #total deaths
            total_deaths.append("How many total {} deaths were reported in {}".format(disease,location))
            total_deaths.append("How many total deaths due to {} were reported in {}".format(disease,location))
            total_deaths.append("What is the total number of deaths due to {} in {}".format(disease,location))
            total_deaths.append("How many total deaths of {} were reported in {}".format(disease,location))
            total_deaths.append("What is the total tally of {} deaths in {}".format(disease,location))
    if len(locations)==0: #list of questions when no location or date is present.
        new_cases.append("How many new {} cases were reported".format(disease))   
        new_cases.append("How many {} cases were reported in the last 24 hours".format(disease))
        new_cases.append("How many new cases were reported") 
        new_cases.append("How many cases were reported in last 24 hours")
        new_deaths.append("How many new deaths due to {} were reported".format(disease))   
        new_deaths.append("How many {} deaths were reported in the last 24 hours".format(disease))
        new_deaths.append("How many new deaths were reported") 
        new_deaths.append("How many deaths were reported in last 24 hours")          
        total_cases.append("How many total cases of {} were reported".format(disease))
        total_cases.append("What is the total tally of {} cases".format(disease))
        total_cases.append("What is the total number of cases")
        total_deaths.append("How many total deaths of {} were reported".format(disease))
        total_deaths.append("What is the total tally of {} deaths".format(disease))
        total_deaths.append("What is the total number of deaths")

    return new_cases,new_deaths,total_cases,total_deaths
    

In [48]:
#Get answer from the QA model for a question.
def get_answer(nlp,question,article,threshold):
    '''
    Returns the answer to the question from the QA model treating the article as the context. 
    INPUTS: 
    nlp: QA model pipeline
    question: question string
    article: context string
    threshold: The minimum score the answer must have to be considered as a legit answer.
    
    The QA model provides us with top-k answers. Currently it is set to top-3 answers. The output of QA model has following attributes:
    1) answer_text 2) score 3) start 4) end
    First we compare the answer text of the top-k answers and merge their scores if the answer-text is the same. We also merge the start as the smallest among all with same answer-text
    (This is done so that if there are two or more occurances of same answer in the context, the model will distribute its score among all the occurances thus reducing the score of an individual occurance. This can make us choose the wrong answer based on higher score.
     Combining the scores of the same answer-text but different starts solves this problem)
    After this scores of each answer is checked. It should be more than the threshold to be considered a legit answer.
    Also, we convert any numeric words like "six" to 6 etc. using the word2number library.
    '''

    input_dict = {}
    input_dict["question"] = question
    input_dict["context"] = article
    
    all_res = nlp(input_dict,handle_impossible_answer=False,topk=3)
    
    all_res = [[res["answer"],res["score"],res["start"]] for res in all_res] #get answer, score and start
    all_res_dict = {}
    for res in all_res:
        if res[0] in all_res_dict.keys():
            all_res_dict[res[0]][0]+=res[1] #add the scores of the same answer but different start
            all_res_dict[res[0]][1] = min(all_res_dict[res[0]][1],res[2]) #get the minimum of two starts
        else:
            all_res_dict[res[0]] = [res[1],res[2]]
    all_res_dict = sorted(all_res_dict.items(),key = lambda x: -x[1][0]) #sort on the basis of score.
    
    #choose the answer with the highest score
    score  = all_res_dict[0][1][0] 
    answer = all_res_dict[0][0]
    start = all_res_dict[0][1][1]
    # print(answer,question,score)
    #check if score is greater then the threshold, if yes, convert the answer into number, return (answer,score,question,start)
    if score>threshold:
        try:
            answer = str(w2n.word_to_num(answer))
        except:
            answer = answer
        # print(answer,question,score)
        return (answer,score,question,start)
    return []


#get the answer based on the score
def get_max(a):
    if len(a)!=0:
        a = sorted(a,key=lambda x:-x[1])
        return a[0]
    return 0

#construct an event out of the givent ingredients.
def construct_event(location,incident,Incident_Type,answer,disease,date):
    event = {}
    unconfirmed_location = []
    # print(location)
    if len(location)!=0:
        unconfirmed_location = location[1]
        
    if len(location)!=0 and location[0].lower() in answer[2].lower():
        location = location[0].lower()
    else:
        location = []
    
    event["Location"] = location
    # event["unconfirmed_location"] = unconfirmed_location
    event["Incident (case or death)"] = incident
    event["Incident Type (new or total)"] = Incident_Type
    event["Number"] = answer[0]
    event["Disease"] = disease
    # event["date"] = date
    event["score"] = answer[1]
    event["start"] = answer[3]
    # "[{'Disease': 'COVID-19', 'Location': 'Gwalior', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'total', 'Number': '10'}, {'Disease': 'COVID-19', 'Location': 'Gwalior', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new ', 'Number': '2'}]"
    return event
    

In [49]:

def get_events(nlp,article,date,disease,location):
    '''Function that gets the events out of the article
    INUPTS:
    1) nlp: QA model pipeline.
    2) article: context for QA model
    3) date: dates found in the article (list of tuple)
    4) location: A location found in the aritcle (tuple) NOTE: only a sigle location.
    
    For each disease in disease_list, First we get the questions for each category (new-cases,new-deaths,etc.) we get best answers for each question, then we get the best
    answers for each category,  finally we convert those answers in to events  (we repeat this process for every disease)'''
    disease_list = [x for x in disease.split(",")]
    all_events = [] #get all the events across multiple diaseases
    if len(location)==0:
        location = []
    else:
        location = [location]
    
    for disease in disease_list:
        #get questions for new-cases,new-deaths,total-cases,total-deaths.etc.

        new_cases_q,new_deaths_q,total_cases_q,total_deaths_q = make_questions(date,location,disease)
        new_case_events = []
        new_death_events = []
        total_case_events = []
        total_death_events = []
        #get the best answer for each question.
        for question in new_cases_q:
            ans = get_answer(nlp,question,article,0.2)
            if len(ans)!=0:
                new_case_events.append(ans)
        for question in new_deaths_q:
            ans = get_answer(nlp,question,article,0.2)
            if len(ans)!=0:
                new_death_events.append(ans)
        for question in total_cases_q:
            ans = get_answer(nlp,question,article,0.2)
            if len(ans)!=0:
                total_case_events.append(ans)
        for question in total_deaths_q:
            ans = get_answer(nlp,question,article,0.2)
            if len(ans)!=0:
                total_death_events.append(ans)
        # print(disease)
        #get the best answer for each category
        new_case = get_max(new_case_events)
        new_death = get_max(new_death_events)
        total_case = get_max(total_case_events)
        total_death = get_max(total_death_events)
        if len(location)!=0:
            input_location = location[0]
        else:
            input_location = []
        # get the events for the best answers of the category.
        if new_case!=0:
            event = construct_event(input_location,"case","new",new_case,disease,date)
            all_events.append(event)
        if new_death!=0:
            event = construct_event(input_location,"death","new",new_death,disease,date)
            all_events.append(event)
        if total_case!=0:
            event = construct_event(input_location,"case","total",total_case,disease,date)
            all_events.append(event)
        if total_death!=0:
            event = construct_event(input_location,"death","total",total_death,disease,date)
            all_events.append(event)
    print("All the events are:"+str(all_events))
    return all_events

In [50]:
def check_in_span(s,spans):
    for span in spans:
        if s>=span[0] and s<=span[1]:
            return True
    return False

#group multiple disease by a single disease using synonyms dict.
def group_disease(events,spans):
    '''Groups multiple synonyms of a single disease into a one.
    synonyms are presennt inthe disease_synonyms.json file'''
    new_events = []
    with open('/home/ubuntu/jushaan/idsp-score/data/05_model_input/disease_synonyms.json',"r") as f:
        synonyms = json.load(f)
    for event in events:
        disease = event["Disease"]
        for key in synonyms.keys():
            if disease in synonyms[key]:
                event["Disease"] = key
        if check_in_span(event["start"],spans)==False:
            new_events.append(event)
    return new_events

def group_disease_remove_score(events):
    events= [{k:v for k,v in event.items() if k!="score"} for event in events]
    events_str = [str(x) for x in events]
    unique_events = list(set(events_str))
    unique_events = [ast.literal_eval(x) for x in unique_events]
    return unique_events

In [51]:

def filter_by_score(events):
    '''Filters events based on scores.
    INPUT: list of events
    First sort the events based on the start and then based on the scores (only the events which have "digit" in event['number'] are kept, rest are removed.
    After sorting, choose only events with unique starts and highest scores for each start.
    (The theory is that no two question should have the answer with same starts, if they do, then pick the one with higher score)'''
    filtered_events = []
    index_list = [[i,int(events[i]["Number"].replace(",","")),events[i]["start"],events[i]["score"]] for i in range(len(events)) if events[i]["Number"].replace(",","").isdigit() ]
    if len(index_list)==0:
        return filtered_events
    index_list = sorted(index_list,key = lambda x: (x[2],-x[3]))

    filtered_events.append(events[index_list[0][0]])
    curr_span = index_list[0][2]
    for i in index_list[1:]:
        if i[2]!=curr_span:
            curr_span = i[2]
            filtered_events.append(events[i[0]])
    return filtered_events


In [52]:
def filter_repeadted_incident(events):
    '''Filter the events which are repeated, for eg. If there are multiple events where location = a and disease = b , incident = case and Incident Type (new or total) = new. they may have differnt "number",
    but, we cannot have multiple events reporting new cases in same location for same disease, so this function filters out those events by keeping only one with the highest score.
    In addition to this, the function also assumes that for a given location and disease, new cases cannot be equall to total cases similarly new death cannot be equall to total deaths.
    Hence, they are also filtered out by keeping the one with the higher score.'''
    # print(events)
    keys = [key for key in events[0].keys() if key!="Incident (case or death)" and key!="Incident Type (new or total)" and key!="Number" and key!="score" and key!="start"]
    new_cases= sorted([event for event in events if event["Incident (case or death)"]=="case" and event["Incident Type (new or total)"]=="new"],key = lambda x: -x["score"])
    new_deaths= sorted([event for event in events if event["Incident (case or death)"]=="death" and event["Incident Type (new or total)"]=="new"],key = lambda x: -x["score"])
    total_cases= sorted([event for event in events if event["Incident (case or death)"]=="case" and event["Incident Type (new or total)"]=="total"],key = lambda x: -x["score"])
    total_deaths= sorted([event for event in events if event["Incident (case or death)"]=="death" and event["Incident Type (new or total)"]=="total"],key = lambda x: -x["score"])

    #filter events where totalcases = newcases for a location and disease
    cases_list = sorted(new_cases+total_cases,key=lambda x: -x["score"])
    keys_cases = [key for key in events[0].keys() if key!="Incident (case or death)" and key!="Incident Type (new or total)" and key!="score" and key!="start"]
    cases_event_list = []
    visited_set = set([])
    for event in cases_list:
        values = [event[k] for k in keys_cases]
        values_str = "".join(map(str,values))
        if values_str not in visited_set:
            cases_event_list.append(event)
            visited_set.add(values_str)
    
    #filter events where totaldeaths = newdeaths for a location and disease
    deaths_list = sorted(new_deaths+total_deaths,key=lambda x: -x["score"])
    keys_deaths = [key for key in events[0].keys() if key!="Incident (case or death)" and key!="Incident Type (new or total)" and key!="score" and key!="start"]
    deaths_event_list = []
    visited_set = set([])
    for event in deaths_list:
        values = [event[k] for k in keys_deaths]
        values_str = "".join(map(str,values))
        if values_str not in visited_set:
            deaths_event_list.append(event)
            visited_set.add(values_str)
                
    #filter events reporting multiple newcases, or newdeaths etc for same location and disease
    events = cases_event_list+ deaths_event_list
    print(events)
    keys = [key for key in events[0].keys() if key!="Incident (case or death)" and key!="Incident Type (new or total)" and key!="number" and key!="score" and key!="start"]
    new_cases= sorted([event for event in events if event["Incident (case or death)"]=="case" and event["Incident Type (new or total)"]=="new"],key = lambda x: -x["score"])
    new_deaths= sorted([event for event in events if event["Incident (case or death)"]=="death" and event["Incident Type (new or total)"]=="new"],key = lambda x: -x["score"])
    total_cases= sorted([event for event in events if event["Incident (case or death)"]=="case" and event["Incident Type (new or total)"]=="total"],key = lambda x: -x["score"])
    total_deaths= sorted([event for event in events if event["Incident (case or death)"]=="death" and event["Incident Type (new or total)"]=="total"],key = lambda x: -x["score"])
    event_list = [new_cases,new_deaths,total_cases,total_deaths]
    events = []
    for i in event_list:
        visited_set = set([])
        for event in i:
            values = [event[k] for k in keys]
            values_str = "".join(map(str,values))
            if values_str not in visited_set:
                events.append(event)
                visited_set.add(values_str) 
    return events

In [53]:
def filter_events(events,spans):
    '''Master function for all filters
    1) Group different disease by using disease synonyms.
    2) Filter events having the same 'start', choose the hightest score one for each start, also filter events where 'number' is not digit
    3) Filter repeated events (eg. multiple events reporting 'new cases' for a single location and disease also, for a single location and disaese, newcases!=totalcases, newdeaths!=totaldeaths, filter them too'''
    temp = group_disease(events,spans)
    # return temp
    if len(temp)!=0:
        temp = filter_by_score(temp)
        # return temp
    if len(temp)!=0:
        temp = filter_repeadted_incident(temp)
    return temp

In [54]:
def Event_Extraction_Using_QA(article):
    '''Master function for event_extraction using QA from an article.
    1) Check if article is relevant, return [] if not
    2) Check if it contains some disease, return [] if not
    3) Get location and dates in the article using ChatbotNER
    4) Get events for each location, combine all events. 
    5) Filter those events'''
    # if isrelevant(article,relevant_model,relevant_tokenizer)==0:
    #     print("not relevant")
    #     return []
    disease,spans = get_disease(article,ner_pipeline)
    if len(disease)==0:
        disease = "some disease"
        
    location = NER(article,"city")
    article = preprocess_location(location,article)
    location= NER(article,"city")
    total_events = []
    date = NER(article,"date")
    if len(location)!=0:
        for i in location:
            events = get_events(qa_pipeline,article,date,disease,i)
            total_events.extend(events)
    else:
        total_events.extend(get_events(qa_pipeline,article,date,disease,[]))
   
    total_events = filter_events(total_events,spans)
    return total_events, disease, location


In [55]:
disease,spans = get_disease(article,ner_pipeline)
print(disease)
if len(disease)==0:
    disease = "some disease"
        
location = NER(article,"city")
article = preprocess_location(location,article)
location= NER(article,"city")

total_events = []
date = NER(article,"date")
print(location)
if len(location)!=0:
    for i in location:
        
        events = get_events(qa_pipeline,article,date,disease,i)
        total_events.extend(events)
else:
    total_events.extend(get_events(qa_pipeline,article,date,disease,[]))
print(total_events)

Dis_list
[{'entity_group': 'bio', 'score': 0.8255292, 'word': 'opd', 'start': 20, 'end': 23}]
prev
{'entity_group': 'bio', 'score': 0.8255292, 'word': 'opd', 'start': 20, 'end': 23}
{'entity_group': 'bio', 'score': 0.8255292, 'word': 'opd', 'start': 20, 'end': 23}
opd
[('mumbai', 'Mumbai')]


/home/marwan/event/lib/python3.8/site-packages/transformers/pipelines/question_answering.py:189: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)


All the events are:[{'Location': 'mumbai', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': '25', 'Disease': 'opd', 'score': 0.9105204939842224, 'start': 8}, {'Location': 'mumbai', 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'new', 'Number': '25', 'Disease': 'opd', 'score': 0.9461569786071777, 'start': 8}, {'Location': 'mumbai', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'total', 'Number': '25', 'Disease': 'opd', 'score': 0.7542154788970947, 'start': 8}, {'Location': 'mumbai', 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'total', 'Number': '25', 'Disease': 'opd', 'score': 0.9353646039962769, 'start': 8}]
[{'Location': 'mumbai', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': '25', 'Disease': 'opd', 'score': 0.9105204939842224, 'start': 8}, {'Location': 'mumbai', 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'new', 'Num

In [56]:
wrong_l=[]

In [57]:
#Evaluation Metrics for Event Extraction.

from scipy.optimize import linear_sum_assignment

def optimal_cost(cost_matrix):
    '''Return the cost of optimal matching by using the bipartite matching algorithm (also known as assingment problem). Objective is to maximize the cost'''
    row_ind, col_ind = linear_sum_assignment(cost_matrix,maximize = True)
    return cost_matrix[row_ind, col_ind].sum()

def isinside(a,b):
    if a.find(b)!=-1 or b.find(a)!=-1:
        return 1
    if is_synonym_disease(a,b)==True or get_synonyms(a,b):
        return 1
    else:
        wrong_l.append((a,b))
        return 0
def get_synonyms(original_disease,answer):
    if(answer=='covid' or answer=='covid19' or answer=='corona' or answer=='coronavirus' or answer=='covid 19' or answer=='covid-19' or answer=='covid-19 '):
        if original_disease in ['covid','covid19','corona','coronavirus','covid 19','covid-19']:
            return True
        return False
    elif(answer=='india' or answer=='country'):
        if original_disease in ['india','country']:
            return True
        return False
    elif (answer=='kerala' or answer=='kerela'):
        if original_disease in ['kerala','kerela']:
            return True
        return False
    elif(answer=='uttarkhand' or answer=='uttarakhand'):
        if original_disease in ['uttarakhand','uttarkhand']:
            return True
        return False
    elif(answer=='0' or answer=='no'):
        if original_disease in ['0','no']:
            return True
        return False
    elif(answer=='chennai' or answer=='madras'):
        # if original_disease in 
        if original_disease in ['chennai','madras']:
            return True
        return True
    elif (answer=='tuberculosis' or answer=='tb'):
        if original_disease in ['tuberculosis','tb']:
            return True
        return False
    elif (answer=='monkeypox' or answer=='monkey pox'):
        if original_disease in ['monkeypox','monkey pox']:
            return True
        return False
    elif(answer=='muzaffarpur' or answer=='muzzaffarpur'):
        if original_disease in ['muzaffarpur','muzzaffarpur']:
            return True
        return False
    elif(answer=='6' or answer=='half a dozen'):
        if original_disease in ['6','half a dozen']:
            return True
        return False
    elif(answer=='telengana' or answer=='telangana'):

        if original_disease in  ['telengana','telangana']:
            return True
        return False
    elif(answer=='gir somanth' or answer=='gir somnath'):
        if original_disease in ['gir somanth','gir somnath']:
            return True
        return False
    elif(answer=='gautam budh nagarup' or answer=='gautam budh nagar'):
        if original_disease in ['gautam budh nagarup','gautam budh nagar']:
            return True
        return False
    elif(answer=='maharastra' or answer=='maharashtr' or answer =='maharashtra' or answer=='maharasthra'):
        if original_disease in  ['maharashtra','maharastra','maharashtr']:
            return True
        return False
    elif(answer=='tamilnadu' or answer=='tamil nadu'):
        if original_disease in  ['tamilnadu','tamil nadu']:
            return True
        return False
    elif(answer=='uttarpradesh' or answer=='uttar pradesh'):
        if original_disease in  ['uttarpradesh','uttar pradesh']:
            return True
        return False
    elif(answer=='food poision' or answer=='food poisoning'):
        if original_disease in ['food poision','food poisoning']:
            return True
        return False
    elif(answer=='gastroentertis' or answer=='gastroenteritis'):
        if original_disease in ['gastroentertis','gastroenteritis']:
            return True
        return False
    elif(answer=='diarrhea' or answer=='diarrheoa'):
        if original_disease in ['diarrhea','diarrheoa']:
            return True
        return False
    # elif(answer==)
        
    else:
        return False
def is_synonym_disease(original_disease,unique_disease):
    
    if original_disease and unique_disease in ['AES','Acute Encephalitis Syndrome','aes']:
        return True
    
    if original_disease and unique_disease in ['African Swine Flu','African Swine Fever']:
        return True
    if original_disease and unique_disease in ['COVID-19','COVID', 'COVID-19', 'COVID-19 ','Corona', 'Corona ', 'Coronavirus', 'Coronavirus ', 'Covid', 'Covid ', 'Covid-19','corona','corona', 'coronavirus']:
        return True
    
    if original_disease and unique_disease in ['H1N1 Virus ', 'H1N1 flu']:
        return True
    if original_disease and unique_disease in ['H3N2', 'H3N2 Flu', 'H3N2 Influenza', 'H3N2 Influenza ', 'H3N2 Virus', 'H3N2 Virus ', 'H3N2 virus']:
        return True
        
    if original_disease and unique_disease in ['JE disease', 'JE disease ', 'Japanese Encephalitis']:
        return True
   
    if original_disease and unique_disease in ['Lumpy Disease', 'Lumpy Skin', 'Lumpy Skin Disease', 'Lumpy disease', 'Lumpy skin disease', 'Lumpy virus']:
        return True
    
    
    if original_disease and unique_disease in ['Malaria','Malaria ']:
        return True
        
    if (original_disease and unique_disease in ['Marburg virus', 'Marburg virus ']):
        return True
   
    if original_disease and unique_disease in ['Monkey Pox', 'Monkey pox', 'Monkeypox']:
        return True

    if original_disease and unique_disease in ['Swine Flu', 'Swine flu']:
        return True
        return False
    if original_disease and unique_disease in ['TB','MDR-TB']:
        return True
    else:
        return False  

def Soft_match(a,b,key_list):
    '''Soft match is the fraction of the number of keys in a predicted event matched with the GT event out of the total keys'''
    score = 0
    # print(a,b,len(a),len(b))
    for key in key_list:
        key=str(key)
       
        if a[key]==None or b[key]==None or len(a[key])==0 or len(b[key])==0:
            if a[key]==b[key]:
                score+=1
        else:
            # print(a[key],b[key])
            if isinside(a[key].replace(",","").lower(),b[key].replace(",","").lower())==1:
                # print("1")
                score+=1
            else:
                print(a[key],b[key])
                pass
                # print('0')
    return score/len(key_list)


def Hard_match(a,b,key_list):
    '''Hard-match returns 1 if all the keys match between predicted and gt events, else return 0'''
    for key in key_list:
        if a[key]==None or b[key]==None or len(a[key])==0 or len(b[key])==0:
            if a[key]!=b[key]:
                return 0
        else:
            if isinside(a[key].replace(",","").lower(),b[key].replace(",","").lower())==0:
                print(a[key],b[key])
                return 0
    return 1

def optimal_match_score(N,K,type="soft"):
    '''Master function to get score of optimal-matching between N GT evens and K predicted events.'''
    # print(N,K)
    if N==None or K==None or len(N)==0 or len(K)==0:
        if N==K:
            return 1
        else:
            return 0
    key_list = ["Location","Incident (case or death)","Disease","Number"]
    # key_list=['location','incident','incident_type','number']
    cost_matrix = []
    for i in N:
        if type=="soft":
            # print('hello')
            temp = [Soft_match(i,j,key_list) for j in K]
            # print(i+" "+j)
        if type=="hard":
            temp = [Hard_match(i,j,key_list) for j in K]
        # print(temp)
        cost_matrix.append(temp)
    cost_matrix = np.array(cost_matrix)
    cost = optimal_cost(cost_matrix)
    return cost
# Evaluation Metric Precision, Recall and F1
def PRF1(score,true,pred):
    precision = score*(min(true,pred))/pred
    recall = score*(min(true,pred))/true
    F1 = (2*precision*recall)/(precision+recall)
    return precision,recall,F1

def PRF2(score,true,pred):
    '''Return precision, recall and F1. 
    1) precision = (number of true events)*score/(number of predicted events)
    2) Recall = score
    3) F1 = 2*P*R/(P+R)'''
    precision = score*(true)/pred
    recall = score
    F1 = (2*precision*recall)/(precision+recall)
    return precision,recall,F1
    
    

In [58]:
net_df.reset_index(inplace=True)

In [59]:
net_df.drop(columns=['index'],inplace=True)
net_df.head()

/tmp/ipykernel_23416/3477088236.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net_df.drop(columns=['index'],inplace=True)


,Index,Original_Article,Translated_Article,Events,QA_data,User
0,0,ग्वालियर में 183 संदिग्ध राेगियाें की हुई जांच...,183 suspected patients were investigated in Gw...,"[{'Disease': 'COVID-19', 'Location': 'Gwalior'...",[{'Question': 'How many suspected COVID-19 pat...,Marwan
1,3,Increasing cases of lumpy skin disease in Utta...,Increasing cases of lumpy skin disease in Utta...,"[{'Disease': 'Lumpy skin disease', 'Location':...",[{'Question': 'In which place Lumpy skin disea...,Saba
2,6,Ambikapur News:पुटू खाने से एक ही परिवार के चा...,Ambikapur News:Four people of the same family ...,"[{'Disease': 'Diarrhea', 'Location': 'Ambikapu...",[{'Question': 'How many people fell ill after ...,Saba
3,7,54 Corna New Case In Bilashpur - जिले कोरोना क...,54 new positive cases of Corona in Bilashpur ...,"[{'Disease': 'Corona', 'Location': 'Bilashpur'...",[{'Question': '54 new positive cases of corona...,Saba
4,9,"कोरोना के मामलों में फिर इजाफा, 24 घंटों में 1...","Corona cases increase again, 18,313 new cases ...","[{'Disease': 'Corona', 'Location': 'India', 'I...",[{'Question': 'How many Corona cases have incr...,Saba


In [60]:
with open('/home/ubuntu/jushaan/idsp-score/data/05_model_input/disease_synonyms.json','r') as f:
    disease_synonyms = json.load(f)

# processing gt disease
for i,item in net_df.iterrows():
    for event in item['Events']:
        for key,values in disease_synonyms.items():
            if event['Disease'].lower() in values:
                event['Disease'] = key
                # print(event)

#processing gt numbers - some events do not have numbers.
print(len(net_df))
for i,item in net_df.iterrows():
    event_list = []
    for event in item['Events']:
        if event['Number'] == '':
            continue
        else:
            event['Number'] = event['Number'].replace(',','')
            event_list.append(event)
    net_df['Events'].iloc[i] = event_list
        
            
            



394


/tmp/ipykernel_23416/3392606511.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net_df['Events'].iloc[i] = event_list
/tmp/ipykernel_23416/3392606511.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net_df['Events'].iloc[i] = event_list
/tmp/ipykernel_23416/3392606511.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net_df['Events'].iloc[i] = event_list
/tmp/ipykernel_23416/3392606511.py:22: SettingWithCopyWarning: 
A valu

In [61]:
net_df.head(2)

,Index,Original_Article,Translated_Article,Events,QA_data,User
0,0,ग्वालियर में 183 संदिग्ध राेगियाें की हुई जांच...,183 suspected patients were investigated in Gw...,"[{'Disease': 'covid-19', 'Location': 'Gwalior'...",[{'Question': 'How many suspected COVID-19 pat...,Marwan
1,3,Increasing cases of lumpy skin disease in Utta...,Increasing cases of lumpy skin disease in Utta...,"[{'Disease': 'Lumpy skin disease', 'Location':...",[{'Question': 'In which place Lumpy skin disea...,Saba


In [62]:
# old_df =  pd.read_csv('old_event_extraction.csv')
# old_df.drop(columns=['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1','Unnamed: 0.1.1.1'],inplace=True)
# old_df.head()


# for i, item in old_df.iterrows():
#     article = item['headline'] + ' '+ item['article']
#     true_labels =  item['cleaned true labels']
#     true_labels = ast.literal_eval(true_labels)
#     old_df['cleaned true labels'].iloc[i] = true_labels

In [63]:
from tqdm import tqdm

true_labels = []
len_predictions=[]
predictions = []
disease_list = []
locations_list = []
count=0
for i, row in tqdm(net_df.iterrows()):
    count=count+1
    print(count)
    if(count<150):
        continue
    true_labels.append(row['Events'])
    pred_labels, diseases, locations = Event_Extraction_Using_QA(row['Translated_Article'])
    
    predictions.append(pred_labels)
    len_predictions.append(len(pred_labels))
    disease_list.append(diseases)
    locations_list.append(locations)
   

0it [00:00, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


/home/marwan/event/lib/python3.8/site-packages/transformers/pipelines/question_answering.py:189: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)


Dis_list
[{'entity_group': 'bio', 'score': 0.82311547, 'word': 'corona', 'start': 18, 'end': 24}, {'entity_group': 'bio', 'score': 0.99672407, 'word': 'covid', 'start': 57, 'end': 62}, {'entity_group': 'bio', 'score': 0.99631315, 'word': 'covid', 'start': 117, 'end': 122}]
prev
{'entity_group': 'bio', 'score': 0.82311547, 'word': 'corona', 'start': 18, 'end': 24}
{'entity_group': 'bio', 'score': 0.82311547, 'word': 'corona', 'start': 18, 'end': 24}
{'entity_group': 'bio', 'score': 0.82311547, 'word': 'corona', 'start': 18, 'end': 24}
{'entity_group': 'bio', 'score': 0.99672407, 'word': 'covid', 'start': 57, 'end': 62}
{'entity_group': 'bio', 'score': 0.99672407, 'word': 'covid', 'start': 57, 'end': 62}
{'entity_group': 'bio', 'score': 0.99631315, 'word': 'covid', 'start': 117, 'end': 122}


150it [00:08, 17.60it/s]

...

All the events are:[{'Location': 'nagaland', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': '3', 'Disease': "nagaland's covid", 'score': 0.9997738003730774, 'start': 0}, {'Location': 'nagaland', 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'new', 'Number': '3', 'Disease': "nagaland's covid", 'score': 0.9979507327079773, 'start': 0}, {'Location': 'nagaland', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'total', 'Number': '31,938', 'Disease': "nagaland's covid", 'score': 0.9999188184738159, 'start': 44}, {'Location': 'nagaland', 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'total', 'Number': '31,938', 'Disease': "nagaland's covid", 'score': 0.9956998825073242, 'start': 44}]
[{'Location': 'nagaland', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'total', 'Number': '31,938', 'Disease': "nagaland's covid", 'score': 0.9999188184738159, 'start': 44}, {'Location': 

393it [3:08:00, 17.67s/it]

All the events are:[{'Location': [], 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': 'lakhs', 'Disease': 'lumpy skin disease', 'score': 0.9541721343994141, 'start': 38}, {'Location': [], 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'new', 'Number': 'Over 7,300', 'Disease': 'lumpy skin disease', 'score': 0.8479270339012146, 'start': 98}, {'Location': [], 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'total', 'Number': '7,300', 'Disease': 'lumpy skin disease', 'score': 0.8193536996841431, 'start': 103}, {'Location': [], 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'total', 'Number': '7,300', 'Disease': 'lumpy skin disease', 'score': 0.972234308719635, 'start': 103}]
[{'Location': [], 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'total', 'Number': '7,300', 'Disease': 'lumpy skin disease', 'score': 0.972234308719635, 'start': 103}]
394
Dis_list
[{'entity_group'

394it [3:08:44, 28.74s/it]

All the events are:[{'Location': 'month', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': '46', 'Disease': 'chikungunya', 'score': 0.9999732971191406, 'start': 87}, {'Location': 'month', 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'new', 'Number': '46', 'Disease': 'chikungunya', 'score': 0.9999603033065796, 'start': 87}, {'Location': 'month', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'total', 'Number': '70', 'Disease': 'chikungunya', 'score': 0.9999784231185913, 'start': 199}, {'Location': 'month', 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'total', 'Number': '70', 'Disease': 'chikungunya', 'score': 0.9999241828918457, 'start': 199}, {'Location': 'month', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': '46', 'Disease': 'dengue', 'score': 0.9997429251670837, 'start': 87}, {'Location': 'month', 'Incident (case or death)': 'death', 'Inciden

In [64]:
import csv

# Example lists of predictions and real labels


# Combine predictions and real labels into a list of tuples
data = list(zip(predictions, true_labels))

# Specify the CSV file path
csv_file_path = 'predictions_and_labels(150-394)_op=0.3.csv'

# Write the data to the CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header
    writer.writerow(['Prediction', 'Real Label'])
    
    # Write data rows
    writer.writerows(data)

print(f"CSV file '{csv_file_path}' has been created with predictions and real labels.")


CSV file 'predictions_and_labels(150-394)_op=0.3.csv' has been created with predictions and real labels.


In [ ]:
from tqdm import tqdm

true_labels = []
len_predictions=[]
predictions = []
disease_list = []
locations_list = []
count=0
for i, row in tqdm(net_df.iterrows()):
    count=count+1
    if(count<150):
        continue
    true_labels.append(row['Events'])
    pred_labels, diseases, locations = Event_Extraction_Using_QA(row['Translated_Article'])
    
    predictions.append(pred_labels)
    len_predictions.append(len(pred_labels))
    disease_list.append(diseases)
    locations_list.append(locations)
   

In [29]:
import csv

# Example lists of predictions and real labels


# Combine predictions and real labels into a list of tuples
data = list(zip(predictions, true_labels))

# Specify the CSV file path
csv_file_path = 'predictions_and_labels(150-394)_op=0.2_baseline.csv'

# Write the data to the CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header
    writer.writerow(['Prediction', 'Real Label'])
    
    # Write data rows
    writer.writerows(data)

print(f"CSV file '{csv_file_path}' has been created with predictions and real labels.")


CSV file 'predictions_and_labels(150-394)_op=0.2_baseline.csv' has been created with predictions and real labels.


In [69]:
import csv
baseline_len__truelist=[]

for i in true_labels:
    baseline_len__truelist.append(len(i))
# Specify the CSV file path
csv_file_path = 'NER_predictions_baseline_4_with_application_of_others.csv'

# Lists to store data from columns
baseline_predictions = predictions
baseline_true = true_labels
baseline_locations_list= locations_list
baseline_disease_list=disease_list
baseline_len__pred_list=len_predictions

# baseline_len__truelist=len()

rows = zip(contexts,baseline_predictions, baseline_true, baseline_locations_list, baseline_disease_list, baseline_len__pred_list,baseline_len__truelist,curr_soft,curr_hard)

# Open the CSV file in write mode
with open(csv_file_path, mode='w', newline='') as file:
    # Create a CSV writer
    writer = csv.writer(file)

    # Write the header row (optional)
    writer.writerow(['Contexts','Baseline Predictions', 'Baseline True Labels', 'Baseline Locations', 'Baseline Diseases', 'Baseline Lengths','True_lengths',"Soft_score","Hard_score"])

    # Write the data rows
    writer.writerows(rows)

print(f'The data has been written to {csv_file_path}')
    
    


The data has been written to NER_predictions_baseline_4_with_application_of_others.csv


In [94]:
# print(baseline_predictions)
print(baseline_disease_list)

['covid - 19,covid - 19 infected,covid-19', 'lumpy skin disease', 'diarrhea,vomiting', 'corona', 'corona,corona virus infection', 'corona,co', 'corona virus,corona infection', 'corona', 'corona', 'lumpy skin disease', 'corona,covid 19', 'covid - 19,covid-19', 'corona infection', 'covid-19,covid - 19 infection', 'encephalitis,chamki fever,aes,a,fever', 'encephalitis,a,acute encephalitis syndrome,aes', 'corona infected,corona infection', 'covid-19,covid - 19,corona virus infection', 'corona', 'cold,influenza infection,flu,fever,covid - 19,##n,cough,covid-19,influenza', 'deaths,covid - 19,covid-19,coronavirus', 'covid - 19,covid-19', 'diarrhea,vomiting', 'covid - 19,covid-19', 'lumpy skin disease', '##oid,viral infections,malaria,ty,dengue,covid - 19,chikungunya,covid-19', 'covid - 19,covid-19,coronavirus', '- tb,m,pain,tb,mdr - tb', 'covid - 19,corona infected,covid-19']


In [70]:
print(baseline_locations_list)

[[('gwalior', 'Gwalior')], [('uttar pradesh', 'Uttar Pradesh'), ('uttarakhand', 'Uttarakhand')], [('ambikapur', 'Surguja'), ('chhattisgarh', 'Chhattisgarh'), ('puttu', 'Puttur')], [('bilashpur', 'Bilaspur')], [('delhi', 'Delhi'), ('punjab', 'Punjab'), ('kesari', 'Kesaria')], [('chhattisgarh', 'Chhattisgarh')], [('delhi', 'Delhi'), ('maharashtra', 'Maharashtra'), ('nagpur', 'Nagpur')], [('karnal', 'Karnal'), ('haryana', 'Haryana')], [('chhattisgarh', 'Chhattisgarh')], [('nagaur', 'Nagaur')], [('delhi', 'Delhi')], [('delhi', 'Delhi')], [('kaithal', 'Kaithal')], [('kaithal', 'Kaithal')], [('kundan', 'Kunda'), ('muzaffarpur', 'Muzaffarpur'), ('vaishali', 'Vaishali'), ('bihar', 'Bihar')], [('muzaffarpur', 'Muzaffarpur'), ('kundan', 'Kunda'), ('vaishali', 'Vaishali'), ('bihar', 'Bihar')], [('rewari', 'Rewari')], [], [('kanpur', 'Kanpur')], [('mumbai', 'Mumbai'), ('maharashtra', 'Maharashtra'), ('nagpur', 'Nagpur'), ('pune', 'Pune'), ('karnataka', 'Karnataka')], [('bihar', 'Bihar')], [('chhat

In [30]:
contexts=[]
for i,row in tqdm(net_df.iterrows()):
    contexts.append(row['Translated_Article'])
contexts=contexts[149:]

394it [00:00, 14891.65it/s]


In [31]:
curr_soft=[]
curr_hard=[]

def score_calculator(true_labels,predictions):
    total_score = 0
    total_packets_pred= 0
    total_packets_true = 0
    total_score_hard = 0

    
    for i in range(len(predictions)):
        breakage = 0
        for event in true_labels[i]:
            if 'Location' not in event.keys():
                breakage =1
                continue
            if 'india' in event['Location'].lower():
                breakage =1
        if breakage==1:
            continue
        curr_score = optimal_match_score(true_labels[i],predictions[i],"soft")
        curr_score_hard = optimal_match_score(true_labels[i],predictions[i],"hard")
        # if curr_score_hard != len(predictions[i]):
        #     print("article")
        #     print(final_results_data['Article'].iloc[i])
        #     print("len of true and pred")
        #     print(len(true_labels[i]),len(predictions[i]))
        #     print("true")
            # for item in true_labels[i]:
            #     for key in item.keys():
            #         print(key,item[key],'\n')
            # print("pred")
            # for item in predictions[i]:
            #     for key in item.keys():
            #         print(key,item[key],'\n')
            # print("current score and hard score")
            # print(curr_score,curr_score_hard)
        
        total_packets_pred+=max(1,len(predictions[i]))
        total_packets_true+=max(1,len(true_labels[i]))
        total_score +=curr_score
        total_score_hard+=curr_score_hard
        curr_soft.append(total_score/total_packets_true)
        curr_hard.append(total_score_hard/total_packets_true)
        print(total_score/total_packets_true)
        print(total_score_hard/total_packets_true)


    cumulative_score = total_score/total_packets_true
    cumulative_score_hard = total_score_hard/total_packets_true
    p,r,f = PRF2(cumulative_score,total_packets_true,total_packets_pred)
    print("Average Soft-match score: ",cumulative_score)
    print("Soft Precision: ",p)
    print("Soft Recall: ",r)
    print("Soft F1: ",f)

    print("----------------------")

    p,r,f = PRF2(cumulative_score_hard,total_packets_true,total_packets_pred)
    print("Average Hard-match score: ",cumulative_score_hard)
    print("Hard Precision: ",p)
    print("Hard Recall: ",r)
    print("Hard F1: ",f)



In [34]:
#extraction using baseline with op=0.2 as the operating point for 150-400 articles with applying others
score_calculator(true_labels,predictions)

case death
death case
0.75
0.0
0.8333333333333334
0.3333333333333333
0.875
0.5
case death
628 19,32,026
case death
628 26,256
death case
3 628
3 6
death case
3 26,256
1932026 628
case death
case death
1932026 26,256
death case
26256 628
death case
26256 19,32,026
case death
628 19,32,026
case death
death case
3 6
death case
3 26,256
1932026 628
case death
case death
death case
death case
0.90625
0.625
11 303
death case
1 303
11 303
death case
0.75
0.5454545454545454
146 81,34,150
case death
146 1,48,400
death case
0 146
death case
8134150 146
case death
8134150 1,48,400
death case
148400 146
death case
148400 81,34,150
146 81,34,150
case death
death case
death case
8134150 146
case death
death case
death case
0.75
0.6
0.765625
0.625
2 16
2 16
0.7794117647058824
0.6470588235294118
Thane maharashtra
198 7,11,313
death case
0 198
Thane maharashtra
death case
0 7,11,313
711313 198
Thane maharashtra
Thane maharashtra
death case
Thane maharashtra
711313 198
Thane maharashtra
0.75
0.6
Salem n

In [32]:
#extraction using baseline with op=0.2 as the operating point for 150-400 articles without applying others
score_calculator(true_labels,predictions)

case death
death case
0.75
0.0
0.8333333333333334
0.3333333333333333
0.875
0.5
case death
628 19,32,026
case death
628 26,256
death case
3 628
3 6
death case
3 26,256
1932026 628
case death
case death
1932026 26,256
death case
26256 628
death case
26256 19,32,026
case death
628 19,32,026
case death
death case
3 6
death case
3 26,256
1932026 628
case death
case death
death case
death case
0.90625
0.625
11 303
death case
1 303
11 303
death case
0.75
0.5454545454545454
146 81,34,150
case death
146 1,48,400
death case
0 146
death case
8134150 146
case death
8134150 1,48,400
death case
148400 146
death case
148400 81,34,150
146 81,34,150
case death
death case
death case
8134150 146
case death
death case
death case
0.75
0.6
0.75
0.5625
2 16
2 16
0.7647058823529411
0.5882352941176471
Thane maharashtra
198 7,11,313
death case
0 198
Thane maharashtra
death case
0 7,11,313
711313 198
Thane maharashtra
Thane maharashtra
death case
Thane maharashtra
711313 198
Thane maharashtra
0.7375
0.55
Salem n

In [105]:
#extraction using baseline with op=0.2 as the operating point for 150 articles with applying others
score_calculator(true_labels,predictions)

2 10
2 10
0.5
0.5
case death
19404 321
case death
0.5
0.5
0.4
0.4
0.5
0.5
183 1,469
1469 183
183 1,469
1469 183
0.625
0.625
0.6666666666666666
0.6666666666666666
48184 5
48184 5
0.6363636363636364
0.6363636363636364
0.6666666666666666
0.6666666666666666
0.6923076923076923
0.6923076923076923
case death
400 1
1960 400
case death
case death
1960 400
case death
0.7166666666666667
0.6666666666666666
death case
1 400
1960 400
death case
1960 400
0.6527777777777778
0.6111111111111112
0.6578947368421053
0.5789473684210527
0.625
0.55
0.5208333333333334
0.4583333333333333
0.54
0.48
41 200
200 41
41 200
200 41
0.5740740740740741
0.5185185185185185
0.5892857142857143
0.5357142857142857
case death
104 2
death case
2 104
case death
death case
0.6166666666666667
0.5666666666666667
death case
2 174
death case
0.609375
0.5625
Chattisgarh chhattisgarh
Chattisgarh chhattisgarh
case death
Chattisgarh chhattisgarh
death case
Chattisgarh chhattisgarh
1700 360
case death
1700 3
death case
1 360
1 3
Chattisga

In [103]:
#extraction using baseline with op=0.2 as the operating point for 150 articles without applying others
score_calculator(true_labels,predictions)

2 10
2 10
0.5
0.5
case death
19404 321
case death
0.5
0.5
0.4
0.4
0.5
0.5
183 1,469
1469 183
183 1,469
1469 183
0.625
0.625
covid-19 corona infection
covid-19 corona infection
0.6388888888888888
0.5555555555555556
48184 5
48184 5
0.6136363636363636
0.5454545454545454
0.6458333333333334
0.5833333333333334
0.6730769230769231
0.6153846153846154
case death
400 1
1960 400
case death
case death
1960 400
case death
0.7
0.6
death case
1 400
1960 400
death case
1960 400
0.6388888888888888
0.5555555555555556
0.6447368421052632
0.5263157894736842
0.6125
0.5
0.5104166666666666
0.4166666666666667
covid-19 corona infected
covid-19 corona infected
0.52
0.4
41 200
200 41
41 200
200 41
0.5555555555555556
0.4444444444444444
H3N2 Influenza  influenza infection
H3N2 Influenza  influenza infection
0.5625
0.42857142857142855
case death
104 2
death case
2 104
case death
death case
0.5916666666666667
0.4666666666666667
Acute Diarrhoeal Disease (including acute gastroenteritis) vomiting
death case
Acute Diarrh

In [71]:
#extraction using fine-tuned with op=0.3 as the operating point for 150 articles with applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Ambikapur chhattisgarh
0.8
0.4
0.8333333333333334
0.5
183 1,469
1469 183
183 1,469
1469 183
0.875
0.625
case death
case death
0.8611111111111112
0.5555555555555556
48184 5
Karnal haryana
case death
48184 12
Karnal haryana
case death
48184 47575
Karnal haryana
case death
5 12
5 48184
Karnal haryana
case death
48184 5
Karnal haryana
Karnal haryana
Karnal haryana
5 48184
Karnal haryana
0.8863636363636364
0.6363636363636364
0.8958333333333334
0.6666666666666666
0.9038461538461539
0.6923076923076923
case death
400 1
400 1960
1960 400
case death
case death
400 1960
1960 400
case death
0.9166666666666666
0.7333333333333333
400 1960
death case
1 400
death case
1960 400
400 1960
death case
death case
1960 400
0.875
0.7222222222222222
0.868421052631579
0.

In [69]:
#extraction using fine-tuned with op=0.3 as the operating point for 150 articles without applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Acute Diarrhoeal Disease (including acute gastroenteritis) vomiting
Ambikapur chhattisgarh
0.75
0.4
0.7916666666666666
0.5
183 1,469
1469 183
183 1,469
1469 183
0.84375
0.625
case death
covid-19 corona infection
case death
0.8055555555555556
0.5555555555555556
48184 5
Karnal haryana
case death
48184 12
Karnal haryana
case death
48184 47575
Karnal haryana
case death
5 12
5 48184
Karnal haryana
case death
48184 5
Karnal haryana
Karnal haryana
Karnal haryana
5 48184
Karnal haryana
0.8409090909090909
0.6363636363636364
0.8541666666666666
0.6666666666666666
0.8653846153846154
0.6923076923076923
case death
400 1
400 1960
1960 400
case death
case death
400 1960
1960 400
case death
0.8833333333333333
0.7333333333333333
400 1960
death case
1 400
death ca

In [35]:
#extraction using fine-tuned with 0.92 as the operating point for 150 articles with applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Ambikapur chhattisgarh
0.8
0.4
0.8333333333333334
0.5
183 1,469
1469 183
183 1,469
1469 183
0.875
0.625
case death
case death
0.8611111111111112
0.5555555555555556
48184 5
5 48184
48184 5
5 48184
0.8863636363636364
0.6363636363636364
0.8958333333333334
0.6666666666666666
0.9038461538461539
0.6923076923076923
400 1960
1960 400
400 1960
1960 400
0.9166666666666666
0.7333333333333333
400 1960
death case
1 400
death case
1960 400
400 1960
death case
death case
1960 400
0.875
0.7222222222222222
0.868421052631579
0.6842105263157895
Bihar muzaffarpur
case death
12 10
Bihar muzaffarpur
0.875
0.7
case death
Muzaffarpur kundan
Muzaffarpur kundan
death case
case death
12 10
Muzaffarpur kundan
2 10
Muzzaffarpur kundan
death case
case death
Muzaffarpur kunda

In [33]:
#extraction using fine-tuned with 0.92 as the operating point for 150 articles without applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Acute Diarrhoeal Disease (including acute gastroenteritis) vomiting
Ambikapur chhattisgarh
0.75
0.4
0.7916666666666666
0.5
183 1,469
1469 183
183 1,469
1469 183
0.84375
0.625
case death
covid-19 corona infection
case death
0.8055555555555556
0.5555555555555556
48184 5
5 48184
48184 5
5 48184
0.8409090909090909
0.6363636363636364
0.8541666666666666
0.6666666666666666
0.8653846153846154
0.6923076923076923
400 1960
1960 400
400 1960
1960 400
0.8833333333333333
0.7333333333333333
400 1960
death case
1 400
death case
1960 400
400 1960
death case
death case
1960 400
0.8472222222222222
0.7222222222222222
covid-19 covid - 19 infection
0.8289473684210527
0.6842105263157895
Bihar muzaffarpur
case death
AES encephalitis
12 10
AES encephalitis
Bihar muzaffa

In [94]:
#extraction using fine-tuned with 0.9 as the operating point for 150 articles with applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Ambikapur chhattisgarh
0.8
0.4
0.8333333333333334
0.5
183 1,469
1469 183
183 1,469
1469 183
0.875
0.625
case death
case death
0.8611111111111112
0.5555555555555556
48184 5
5 48184
48184 5
5 48184
0.8863636363636364
0.6363636363636364
0.8958333333333334
0.6666666666666666
0.9038461538461539
0.6923076923076923
400 1960
1960 400
400 1960
1960 400
0.9166666666666666
0.7333333333333333
400 1960
death case
1 400
death case
1960 400
400 1960
death case
death case
1960 400
0.875
0.7222222222222222
0.868421052631579
0.6842105263157895
Bihar muzaffarpur
case death
12 10
Bihar muzaffarpur
0.875
0.7
case death
Muzaffarpur kundan
Muzaffarpur kundan
death case
case death
12 10
Muzaffarpur kundan
2 10
Muzzaffarpur kundan
death case
case death
Muzaffarpur kunda

In [92]:
#extraction using fine-tuned with 0.9 as the operating point for 150 articles without applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Acute Diarrhoeal Disease (including acute gastroenteritis) vomiting
Ambikapur chhattisgarh
0.75
0.4
0.7916666666666666
0.5
183 1,469
1469 183
183 1,469
1469 183
0.84375
0.625
case death
covid-19 corona infection
case death
0.8055555555555556
0.5555555555555556
48184 5
5 48184
48184 5
5 48184
0.8409090909090909
0.6363636363636364
0.8541666666666666
0.6666666666666666
0.8653846153846154
0.6923076923076923
400 1960
1960 400
400 1960
1960 400
0.8833333333333333
0.7333333333333333
400 1960
death case
1 400
death case
1960 400
400 1960
death case
death case
1960 400
0.8472222222222222
0.7222222222222222
covid-19 covid - 19 infection
0.8289473684210527
0.6842105263157895
Bihar muzaffarpur
case death
AES encephalitis
12 10
AES encephalitis
Bihar muzaffa

In [31]:
#extraction using fine-tuned with 0.3 as the operating point for 150 articles without applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Acute Diarrhoeal Disease (including acute gastroenteritis) vomiting
Ambikapur chhattisgarh
0.75
0.4
0.7916666666666666
0.5
183 1,469
1469 183
183 1,469
1469 183
0.84375
0.625
case death
covid-19 corona infection
case death
0.8055555555555556
0.5555555555555556
48184 5
Karnal haryana
case death
48184 12
Karnal haryana
case death
48184 47575
Karnal haryana
case death
5 12
5 48184
Karnal haryana
case death
48184 5
Karnal haryana
Karnal haryana
Karnal haryana
5 48184
Karnal haryana
0.8409090909090909
0.6363636363636364
0.8541666666666666
0.6666666666666666
0.8653846153846154
0.6923076923076923
case death
400 1
400 1960
1960 400
case death
case death
400 1960
1960 400
case death
0.8833333333333333
0.7333333333333333
400 1960
death case
1 400
death ca

In [33]:
#extraction using fine-tuned with 0.3 as the operating point for 150 articles with applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Ambikapur chhattisgarh
0.8
0.4
0.8333333333333334
0.5
183 1,469
1469 183
183 1,469
1469 183
0.875
0.625
case death
case death
0.8611111111111112
0.5555555555555556
48184 5
Karnal haryana
case death
48184 12
Karnal haryana
case death
48184 47575
Karnal haryana
case death
5 12
5 48184
Karnal haryana
case death
48184 5
Karnal haryana
Karnal haryana
Karnal haryana
5 48184
Karnal haryana
0.8863636363636364
0.6363636363636364
0.8958333333333334
0.6666666666666666
0.9038461538461539
0.6923076923076923
case death
400 1
400 1960
1960 400
case death
case death
400 1960
1960 400
case death
0.9166666666666666
0.7333333333333333
400 1960
death case
1 400
death case
1960 400
400 1960
death case
death case
1960 400
0.875
0.7222222222222222
0.868421052631579
0.

In [68]:
#extraction using baseline for 150 articles with applying others
score_calculator(true_labels,predictions)

2 10
2 10
0.5
0.5
Uttarakhand uttar pradesh
case death
19404 321
Uttarkhand uttar pradesh
death case
321 19404
Uttarakhand uttar pradesh
case death
Uttarkhand uttar pradesh
0.6875
0.5
0.55
0.4
0.625
0.5
183 1,469
1469 183
183 1,469
1469 183
0.71875
0.625
0.75
0.6666666666666666
48184 5
48184 5
0.7045454545454546
0.6363636363636364
0.7291666666666666
0.6666666666666666
0.75
0.6923076923076923
case death
400 1
1960 400
case death
case death
1960 400
case death
0.7666666666666667
0.6666666666666666
death case
1 400
1960 400
death case
1960 400
0.6944444444444444
0.6111111111111112
0.6973684210526315
0.5789473684210527
Bihar muzaffarpur
12 5
Bihar muzaffarpur
0.6875
0.55
0.5729166666666666
0.4583333333333333
0.59
0.48
41 200
200 41
41 200
200 41
0.6203703703703703
0.5185185185185185
0.6339285714285714
0.5357142857142857
case death
104 2
death case
2 104
case death
death case
0.6583333333333333
0.5666666666666667
death case
2 174
death case
0.6484375
0.5625
Chattisgarh chhattisgarh
Chattisg

In [64]:
#extraction using baseline for 150 articles 
score_calculator(true_labels,predictions)

2 10
2 10
0.5
0.5
Uttarakhand uttar pradesh
case death
19404 321
Uttarkhand uttar pradesh
death case
321 19404
Uttarakhand uttar pradesh
case death
Uttarkhand uttar pradesh
0.6875
0.5
0.55
0.4
0.625
0.5
183 1,469
1469 183
183 1,469
1469 183
0.71875
0.625
covid-19 corona infection
covid-19 corona infection
0.7222222222222222
0.5555555555555556
48184 5
48184 5
0.6818181818181818
0.5454545454545454
0.7083333333333334
0.5833333333333334
0.7307692307692307
0.6153846153846154
case death
400 1
1960 400
case death
case death
1960 400
case death
0.75
0.6
death case
1 400
1960 400
death case
1960 400
0.6805555555555556
0.5555555555555556
0.6842105263157895
0.5263157894736842
Bihar muzaffarpur
12 5
Bihar muzaffarpur
0.675
0.5
0.5625
0.4166666666666667
covid-19 corona infected
covid-19 corona infected
0.57
0.4
41 200
200 41
41 200
200 41
0.6018518518518519
0.4444444444444444
H3N2 Influenza  influenza infection
H3N2 Influenza  influenza infection
0.6071428571428571
0.42857142857142855
case death
10

In [38]:
#extraction using best_models for 150 articles with applying others and removing location
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.8333333333333333
0.5
case death
19404 321
death case
321 19404
case death
death case
0.9166666666666666
0.75
case death
case death
0.8666666666666666
0.6
0.8888888888888888
0.6666666666666666
183 1,469
1469 183
183 1,469
1469 183
0.9166666666666666
0.75
case death
case death
0.8888888888888888
0.6666666666666666
48184 5
case death
48184 12
case death
48184 47575
case death
5 12
5 48184
case death
48184 5
case death
case death
case death
5 48184
case death
0.9090909090909091
0.7272727272727273
0.9166666666666666
0.75
0.9230769230769231
0.7692307692307693
case death
400 1
400 1960
1960 400
case death
case death
400 1960
1960 400
case death
0.9333333333333333
0.8
400 1960
death case
1 400
death case
1960 400
400 1960
death case
death case
1960 400
0.8888888888888888
0.7777777777777778
0.8947368421052632
0.7894736842105263
case death
12 10
case death
0.9
0.8
case death
death case
case death
12 10
2 10
death case
case death
death case
case death
2 10
de

In [35]:
#extraction using best_models for 150 articles with applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Ambikapur chhattisgarh
0.8
0.4
0.8333333333333334
0.5
183 1,469
1469 183
183 1,469
1469 183
0.875
0.625
case death
case death
0.8611111111111112
0.5555555555555556
48184 5
Karnal haryana
case death
48184 12
Karnal haryana
case death
48184 47575
Karnal haryana
case death
5 12
5 48184
Karnal haryana
case death
48184 5
Karnal haryana
Karnal haryana
Karnal haryana
5 48184
Karnal haryana
0.8863636363636364
0.6363636363636364
0.8958333333333334
0.6666666666666666
0.9038461538461539
0.6923076923076923
case death
400 1
400 1960
1960 400
case death
case death
400 1960
1960 400
case death
0.9166666666666666
0.7333333333333333
400 1960
death case
1 400
death case
1960 400
400 1960
death case
death case
1960 400
0.875
0.7222222222222222
0.868421052631579
0.

In [30]:
#extraction using best_models for 150 articles without applying others
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Acute Diarrhoeal Disease (including acute gastroenteritis) vomiting
Ambikapur chhattisgarh
0.75
0.4
0.7916666666666666
0.5
183 1,469
1469 183
183 1,469
1469 183
0.84375
0.625
case death
covid-19 corona infection
case death
0.8055555555555556
0.5555555555555556
48184 5
Karnal haryana
case death
48184 12
Karnal haryana
case death
48184 47575
Karnal haryana
case death
5 12
5 48184
Karnal haryana
case death
48184 5
Karnal haryana
Karnal haryana
Karnal haryana
5 48184
Karnal haryana
0.8409090909090909
0.6363636363636364
0.8541666666666666
0.6666666666666666
0.8653846153846154
0.6923076923076923
case death
400 1
400 1960
1960 400
case death
case death
400 1960
1960 400
case death
0.8833333333333333
0.7333333333333333
400 1960
death case
1 400
death ca

In [ ]:
# final_results_data  = pd.read_csv('./final_results_only_new.csv')

In [ ]:
net_df.head()

In [ ]:
for i, item in final_results_data.iterrows():
    for j, true_annotation in net_df.iterrows():

        if item['Article'] == true_annotation['Translated_Article']:
            final_results_data['True Labels'].iloc[i] = true_annotation['Events']
            

final_results_data.tail()
    

In [55]:
true_labels = final_results_data['True Labels'].tolist()
predictions = final_results_data['Predicted Labels'].apply(lambda x: eval(x)).tolist()

NameError: name 'final_results_data' is not defined

In [33]:
with open('/home/ubuntu/jushaan/idsp-score/data/05_model_input/disease_synonyms.json','r') as f:
    disease_synonyms = json.load(f)

disease_synonyms['covid-19']+=['corona infection','corona infected','coronavirus infection','Coronavirus']
# processing gt disease
for i,item in enumerate(predictions):
    final_events =[]
    for event in item:
        for key,values in disease_synonyms.items():
            if event['Disease'].lower() in values:
                event['Disease'] = key
            else:
                event['Disease'] = 'others'
        final_events.append(event)
    predictions[i] = final_events

for i,item in enumerate(true_labels):
    final_events =[]
    for event in item:
        for key,values in disease_synonyms.items():
            if event['Disease'].lower() in values:
                event['Disease'] = key
            else:
                event['Disease'] = 'others'
        final_events.append(event)
    true_labels[i] = final_events

        
        
    

In [80]:
print(true_labels)


[[{'Disease': 'others', 'Location': 'Gwalior', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'total', 'Number': '10'}, {'Disease': 'others', 'Location': 'Gwalior', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new ', 'Number': '2'}], [{'Disease': 'others', 'Location': 'Uttarakhand', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': '19404'}, {'Disease': 'others', 'Location': 'Uttarkhand', 'Incident (case or death)': 'death', 'Incident Type (new or total)': 'total', 'Number': '321'}], [{'Disease': 'others', 'Location': 'Ambikapur', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': '4'}], [{'Disease': 'others', 'Location': 'Bilashpur', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': '54'}], [{'Disease': 'others', 'Location': 'India', 'Incident (case or death)': 'case', 'Incident Type (new or total)': 'new', 'Number': '18313'}, {'Disease':

In [87]:
print(len(contexts),len(true_labels),len(predictions),len(disease_list),len(locations_list),len(baseline_predictions),len(baseline_disease_list),len(baseline_locations_list))


394 29 29 29 0 29 58 0


In [95]:
import csv

# Three example lists
#  predictions,disease_list,locations_list

# Zip the three lists together
data = zip(contexts,true_labels,predictions,disease_list,locations_list,baseline_predictions,baseline_disease_list,baseline_locations_list)

# Specify the CSV file path
csv_file_path = 'NER_predictions_finetuned_1.csv'

# Open the CSV file in write mode
with open(csv_file_path, mode='w', newline='') as file:
    # Create a CSV writer
    writer = csv.writer(file)

    # Write the header row (optional)
    writer.writerow(['Contexts', 'True Labels', 'Predicted Labels','disease_list,locations_list','baseline_predictions','baseline_disease_list','baseline_locations_list'])
    # disease_list,locations_list,baseline_predictions,baseline_disease_list,baseline_locations_list
    # Write the data rows
    # writer.writerows(data)
    num_rows = min(len(contexts),len(true_labels),len(predictions),len(disease_list),len(locations_list),len(baseline_predictions),len(baseline_disease_list),len(baseline_locations_list))
    print(num_rows)
    # Write the data rows column-wise
    for i in range(29):
        # print([contexts[i],true_labels[i],predictions[i],disease_list[i],locations_list[i],baseline_predictions[i],baseline_disease_list[i],baseline_locations_list[i]])
        writer.writerow([contexts[i],true_labels[i],predictions[i],disease_list[i],baseline_predictions[i],baseline_disease_list[i],baseline_locations_list[i]])

print(f'The data has been saved to {csv_file_path}')


0
The data has been saved to NER_predictions_finetuned_1.csv


In [78]:
print(data[0])

TypeError: 'zip' object is not subscriptable

In [54]:
true_labels[-2], predictions[-2], final_results_data.iloc[-2]['Article'], final_results_data.iloc[-2]['True Labels']

NameError: name 'final_results_data' is not defined

In [ ]:
final_results_data.to_csv('Monday_updated_results.csv')

In [75]:
#extraction using best_models for 30 articles
score_calculator(true_labels,predictions)

10 183
2 183
2 10
10 183
2 183
2 10
0.875
0.5
case death
19404 321
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
death case
321 19404
case death
Uttarakhand uttar pradesh
Uttarkhand uttar pradesh
0.875
0.5
Ambikapur chhattisgarh
case death
Ambikapur chhattisgarh
0.8
0.4
0.8333333333333334
0.5
183 1,469
1469 183
183 1,469
1469 183
0.875
0.625
case death
case death
0.8611111111111112
0.5555555555555556
48184 5
Karnal haryana
case death
48184 12
Karnal haryana
case death
48184 47575
Karnal haryana
case death
5 12
5 48184
Karnal haryana
case death
48184 5
Karnal haryana
Karnal haryana
Karnal haryana
5 48184
Karnal haryana
0.8863636363636364
0.6363636363636364
0.8958333333333334
0.6666666666666666
0.9038461538461539
0.6923076923076923
case death
400 1
400 1960
1960 400
case death
case death
400 1960
1960 400
case death
0.9166666666666666
0.7333333333333333
400 1960
death case
1 400
death case
1960 400
400 1960
death case
death case
1960 400
0.875
0.7222222222222222
0.868421052631579
0.

In [33]:
#extraction using roberta-large-squad2 for 30 articles
score_calculator(true_labels,predictions)

2 10
2 10
0.5
0.5
Uttarakhand uttar pradesh
case death
19404 321
Uttarkhand uttar pradesh
death case
321 19404
Uttarakhand uttar pradesh
case death
Uttarkhand uttar pradesh
0.6875
0.5
0.55
0.4
0.625
0.5
183 1,469
1469 183
183 1,469
1469 183
0.71875
0.625
0.75
0.6666666666666666
48184 5
48184 5
0.7045454545454546
0.6363636363636364
0.7291666666666666
0.6666666666666666
0.75
0.6923076923076923
case death
400 1
1960 400
case death
case death
1960 400
case death
0.7666666666666667
0.6666666666666666
death case
1 400
1960 400
death case
1960 400
0.6944444444444444
0.6111111111111112
0.6973684210526315
0.5789473684210527
Bihar muzaffarpur
12 5
Bihar muzaffarpur
0.6875
0.55
0.5729166666666666
0.4583333333333333
0.59
0.48
41 200
200 41
41 200
200 41
0.6203703703703703
0.5185185185185185
0.6339285714285714
0.5357142857142857
case death
104 2
death case
2 104
case death
death case
0.6583333333333333
0.5666666666666667
death case
2 174
death case
0.6484375
0.5625
Chattisgarh chhattisgarh
Chattisg

In [ ]:
#extraction using roberta-large-squad2 for 394 articles
score_calculator(true_labels,predictions)

In [64]:
#extraction using best_model for 394 articles
score_calculator(true_labels,predictions)

Average Soft-match score:  0.8139880952380952
Soft Precision:  0.762546468401487
Soft Recall:  0.8139880952380952
Soft F1:  0.7874280230326295
----------------------
Average Hard-match score:  0.5972222222222222
Hard Precision:  0.5594795539033457
Hard Recall:  0.5972222222222222
Hard F1:  0.5777351247600768


In [34]:
#extraction for 30 articles using best_model
score_calculator(true_labels,predictions)

Average Soft-match score:  0.8170731707317073
Soft Precision:  0.7790697674418605
Soft Recall:  0.8170731707317073
Soft F1:  0.7976190476190477
----------------------
Average Hard-match score:  0.5853658536585366
Hard Precision:  0.5581395348837209
Hard Recall:  0.5853658536585366
Hard F1:  0.5714285714285714
